# Char-based text generation with LSTM

In [33]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [34]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'н', 'т', 'и', 'с', 'р', 'л', 'в', '\n', 'д', 'м', 'к', 'ы', ',', 'ь', 'й', 'у', 'п', 'б', 'з', 'г', 'я', 'ч', 'ж', 'х', '!', 'ш', 'ц', '.', 'В', 'Т', 'ю', 'С', '—', 'Н', 'П', 'щ', 'О', 'И', 'К', ':', 'Г', '?', 'Ч', 'М', 'Д', 'Б', 'ф', 'А', 'З', 'э', ';', 'Р', 'Л', 'Я', 'У', '-', 'Е', '…', 'Ф', 'Э', '«', 'Х', '(', ')', '»', 'Ц', 'ъ', 'Ш', 'Ж', 'Щ']


In [35]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [36]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

In [37]:
class TextRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [38]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=85, embedding_size=35, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.167424602508545
 вота срото тесто вом свосто то воте потоке сел стесто восто баде воки се лозот сролодером вевовий соталет сно росте но пато вол веда песта полити сесто тоно поти сено ссро вом сна вем вас вовень пето 
Loss: 2.507268133163452
 вемной дорей веланья
 И торой кора вордой вела,
 Отсе мезденья стом тола вемной
 То рового слава,
 И провова не веденья
 Стора стобронье не в ворой пода варенья,
 Словет вам стобом вал ватонь верной,

Loss: 2.2590175819396974
 раздерной в не ваченья
 В ты сорда слез ведный на пумелей
 В не поренья в сорный праветь сомный престреть пакрен,
 В ты поковь сордолиный был порей провью в собой!
 Ты верной слесть в грасти стил нень
Loss: 2.057433841228485
 пердца в сердца,
 С твой стал свои в собиль,
 В не вроствой стого в поровой в тобой
 В тобой стал на слести ствоей!
 О покольсти провь на в небо нам
 Она мели пороны сердца кровь
 На провь полино пори
Loss: 1.8416294407844545
 пар морей!
 Где соберит воспомовила,
 Ты давы все поворенья
 Востора 

KeyboardInterrupt: 

In [47]:
model.eval()

print(evaluate(
    model,
    char_to_idx,
    idx_to_char,
    temp=0.3,
    prediction_len=700,
    start_text='. '
    )
)

. для кровью истекла:
 Та кровь исчезнуть не могла.
 Как смерч из океанских вод,
 Она из жгучих ран встает,
 Сливаясь в вихре горних сфер
 С твоей, герой Лабэдойер
 (Под бедный стыдливый любви поцелуй!
 
 Пусть старость мне кровь бескля кли.
 Смерть не воскромила —
 Там сперных дворцов,
 Не верить в друга, как в умеч — кок престоле.
 Ты ль покизонах струя
 В убий любви поцелуй!
 
 Не бойся, что Феб отвратит свои взоры,
 О помощи муз не жалей, не тоскуй.
 Что Феб музагет! что парнасские хоры!
 Заменит их первый любви поцелуй!
 
 Созданья мечты, где — облик брону?
 Ток одевы и погибнет уменья
 И пустоты — в тщеславье дев!
 
 Но знаю: ты лишь имя! Надо
 Сойти из облачных дворцов,
 Не верить в дру
